# GLTF 格式教學 Accessor 篇

<a href="https://colab.research.google.com/github/CSP-GD/notes/blob/master/practice/file_format/gltf%E6%A0%BC%E5%BC%8F%E8%A7%A3%E6%9E%90/accessor/accessor.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

[`Open in observablehq`](https://observablehq.com/@toonnyy8/gltf-accessor)

![圖 1. buffers, bufferViews, accessors \[1\]](https://github.com/CSP-GD/notes/raw/master/practice/file_format/gltf%E6%A0%BC%E5%BC%8F%E8%A7%A3%E6%9E%90/accessor/gltfOverview-2.0.0b-accessor.png)

圖 1. buffers, bufferViews, accessors \[1\]

## 簡介

### Accessor 屬性

- bufferView : \<`number`\>  
> 此 Accessor 是從哪個 BufferView 取得數據。

- byteOffset :\<`number`\>  
> 從 BufferView 偏移多少個 byteOffset 的位置開始取數據。

- type : <`string`>  
> 表示一筆數據的類型(count 的單位)  
> `SCALAR` : $1$ 個 componentType 構成  
> `VEC2` : $2$ 個 componentType 構成  
> `VEC3` : $3$ 個 componentType 構成  
> `VEC4` : $4$ 個 componentType 構成  
> `MAT2` : $2*2$ 個 componentType 構成  
> `MAT3` : $3*3$ 個 componentType 構成  
> `MAT4` : $4*4$ 個 componentType 構成  

- componentType : \<`GL Constant of Data Type`\>  
> 表示數據的型別，以下幾種為部分 componentType 代表的型別  
> `5120` : `BYTE`  
> `5121` : `UNSIGNED_BYTE`  
> `5122` : `SHORT`  
> `5123` : `UNSIGNED_SHORT`  
> `5124` : `INT`  
> `5125` : `UNSIGNED_INT`  
> `5126` : `FLOAT`  
  
- count : \<`number`\>  
> 有幾筆數據

- min : \<`type<componentType>`\>
> 數據的最大值

- max : \<`type<componentType>`\>
> 數據的最小值


### BufferView 屬性

- buffer : \<`number`\>  
> 此 BufferView 是從哪個 Buffer 取得數據。

- byteOffset : \<`number`\>  
> 從 Buffer 偏移多少個 byteOffset 的位置開始取數據。

- byteLength : \<`number`\>  
> 要取下多少個 byte。

- byteStride : \<`number`\>  
> 數據交錯擺放時，讓 Accessor 知道取數據的步伐要多少。

- target : \<`34962`|`34963`\>  
> 用來分辨數據的性質為 vertex (target 等於 `34962`，代表 `ARRAY_BUFFER`) 還是 vertex indices (target 等於 `34963`，代表 `ELEMENT_ARRAY_BUFFER`)。

### Buffer 屬性

- byteLength : \<`number`\>  
> 此 Buffer 的大小。

- uri : \<`string`\>  
> bufferData 的位置，也可能用 base64 直接儲存 bufferData。

## 正式開始

### 載入 glTF_tools

In [9]:
!wget https://github.com/CSP-GD/notes/raw/master/practice/file_format/gltf%E6%A0%BC%E5%BC%8F%E8%A7%A3%E6%9E%90/gltf-tools.ipynb -O gltf-tools.ipynb
%run ./gltf-tools.ipynb

--2020-04-08 06:18:33--  https://github.com/CSP-GD/notes/raw/master/practice/file_format/gltf%E6%A0%BC%E5%BC%8F%E8%A7%A3%E6%9E%90/gltf-tools.ipynb
Resolving github.com (github.com)... 140.82.118.3
Connecting to github.com (github.com)|140.82.118.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/CSP-GD/notes/master/practice/file_format/gltf%E6%A0%BC%E5%BC%8F%E8%A7%A3%E6%9E%90/gltf-tools.ipynb [following]
--2020-04-08 06:18:33--  https://raw.githubusercontent.com/CSP-GD/notes/master/practice/file_format/gltf%E6%A0%BC%E5%BC%8F%E8%A7%A3%E6%9E%90/gltf-tools.ipynb
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5840 (5.7K) [text/plain]
Saving to: ‘gltf-tools.ipynb’

gltf-tools.ipynb    100%[==============

## 載入檔案

In [10]:
!wget https://github.com/CSP-GD/notes/raw/master/practice/file_format/gltf%E6%A0%BC%E5%BC%8F%E8%A7%A3%E6%9E%90/accessor/cube.glb -O cube.glb
glb_file = open('./cube.glb', 'rb')
glb_bytes = glb_file.read()
model, buffers = glTF_tools.glb_loader(glb_bytes)

--2020-04-08 06:18:35--  https://github.com/CSP-GD/notes/raw/master/practice/file_format/gltf%E6%A0%BC%E5%BC%8F%E8%A7%A3%E6%9E%90/accessor/cube.glb
Resolving github.com (github.com)... 140.82.118.3
Connecting to github.com (github.com)|140.82.118.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/CSP-GD/notes/master/practice/file_format/gltf%E6%A0%BC%E5%BC%8F%E8%A7%A3%E6%9E%90/accessor/cube.glb [following]
--2020-04-08 06:18:36--  https://raw.githubusercontent.com/CSP-GD/notes/master/practice/file_format/gltf%E6%A0%BC%E5%BC%8F%E8%A7%A3%E6%9E%90/accessor/cube.glb
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1916 (1.9K) [application/octet-stream]
Saving to: ‘cube.glb’

cube.glb            100%[=====

In [11]:
glTF_tools.render_JSON(model)

In [12]:
glTF_tools.render_JSON(model['accessors'])

In [13]:
glTF_tools.render_JSON(model['bufferViews'])

## 參考

1. https://github.com/KhronosGroup/glTF